In [1]:
import pandas as pd
import os
from sklearn.metrics         import confusion_matrix, classification_report
from sklearn.preprocessing   import StandardScaler

path = '/data/home/zhaocj/spn_forcast/df数据整理/../7mer_output/RBIND_7mer'
#path = '/data/home/zhaocj/spn_forcast/df数据整理/../11mer_output/RBIND_11mer'
#path = '../9mer_output/RBIND_9mer'

# test.txt 是X92到X99这几株菌的合并
#path = '../11mer_output/test.txt'

# test2 是每株菌的head
#path = '../11mer_output/test2.txt'

# test3 只是100这株菌
#path = '../11mer_output/test3.txt'

# test4 是每株菌的head, 整理后的数据
#path = '../11mer_output/test4.txt'

# test5 是整理后的数据，是准确的
#path = '../11mer_output/test5.sh'

drug = 'PEN_NM'
drug = 'PEN_MIC'

try:
    df = pd.read_table(path, header=None, names=['SEQ', 'COUNT', 'ID'])
    id_number = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/ID.txt')
    ris = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/ris.txt')
    mic = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/mic.txt')
    sero = pd.read_table('/data/home/zhaocj/seroba/summary.tsv', header=None, names=['ID', 'SEROTYPE', 'COMMENT'])
    sero2 = pd.read_table('sero2.txt')
    sero2['COMMENT'] = 'NaN'
    sero = pd.concat([sero, sero2])
except:
    print('数据导入失败！/(ㄒoㄒ)/~~')
else:
    print('数据导入成功😄！')
    

/data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


数据导入成功😄！


In [13]:
try:
    df = df.pivot_table(index=['ID'], columns='SEQ', values='COUNT', fill_value=0).reset_index()
    
    # 把 df 的ID转换为字符型
    for i in range(df.shape[0]):
        df.iloc[i, 0] = str(df.iloc[i, 0])

    # 选取id_number数据表里有的菌
    df = df.set_index('ID')
    df = df.loc[list(id_number['ID'])].reset_index()
    
    # 选取id_number数据表里有的血清型
    sero = sero.set_index('ID')
    sero = sero.loc[list(id_number['ID'])].reset_index()
    
    
    # 如果drug里面有MIC则合并MIC的表
    if 'MIC' in drug:
        mic = mic[['ID', drug]]
        for i in range(mic.shape[0]):
            mic.iloc[i, 1] = str(mic.iloc[i, 1])
        
        # 如果用血清型就用这个，然后把血清型用dummy来计算
        #df = pd.merge(df, sero.iloc[:,:-1], how = 'left', on=['ID'])
        
        df = pd.merge(df, mic, how = 'left', on=['ID'])
    else:
        ris = ris[['ID', drug]]
        
        # 如果用血清型就用这个，然后把血清型用dummy来计算
        #df = pd.merge(df, sero.iloc[:,:-1], how = 'left', on=['ID'])
        
        df = pd.merge(df, ris, how = 'left', on=["ID"])
    ID = df.iloc[:,0]
    df = df.iloc[:,1:]
    
    
    #print(df.head())
except:
    print('出错了，哪里错了，不知道。')
else:
    print('运气真好，做对了😄！')

运气真好，做对了😄！


In [14]:
# pd.get_dummies(df['PEN_MIC'])

In [15]:
# 血清型作为dammy数据

df_dummies = pd.get_dummies(df['SEROTYPE'], drop_first=True)
df = pd.concat([df.iloc[:,:-2], df_dummies, df.iloc[:,-1]],  axis=1)

In [ ]:
# 进行正态转换后的结果
# 正态转换后和没有正态转换的结果是一样的。

import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split

# Create arrays for the features and the target: X, y
X, y = df.iloc[:,:-1], df.iloc[:,-1]

# Create the training and test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

# 正态转换
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train_scaled, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test_scaled)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

print("accuracy: %f" % (accuracy))




In [16]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split

# Create arrays for the features and the target: X, y
X, y = df.iloc[:,:-1], df.iloc[:,-1]

# Create the training and test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

print("accuracy: %f" % (accuracy))

[15:52:46] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy: 0.559055


In [ ]:
preds

In [ ]:
np.array(y_test)

In [ ]:
np.array(ID.loc[X_test.index])

In [ ]:
# 混淆矩阵
print('混淆矩阵')
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))
print()

In [ ]:
# 导出数据

list_dict = {
    '菌株号': list(np.array(ID.loc[X_test.index])),
    '预测结果': list(preds),
    '实际结果': list(np.array(y_test))
}

pd.DataFrame(list_dict).to_csv(path.split('/')[-2].split('_')[0] + '_' + drug +'.csv')

In [ ]:
# Create the DMatrix from X and y: churn_dmatrix
dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"binary:logistic", "max_depth":3}

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=dmatrix, params=params, 
                    nfold=3, num_boost_round=5, 
                    metrics="error", as_pandas=True, seed=123)